# Introduction to Python & Databases

-----

Previously, we were introduced to SQL and learned how to use SQL to create a schema, insert data into the new schema, how to query these data, and how to modify and delete these data and the schema itself. However, this was all done manually by using a database client tool, specifically the SQLite command line tool. A more efficient mechanisms is to enable automatic database interactions from within a Python program.

In this lesson, we explore how to connect to a [SQLite database][1] from with a Python program. This will build on ideas presented in the previous two lessons. While some of what we do will be SQLite specific, most of what we do will be generic and easily [applied to other databases][2] for which a [Python database interface library][3] has been developed, which is most major SQL based databases. Finally, we will explore how the Pandas library and the DataFrame can be used to simplify these steps.

-----
[1]: https://docs.python.org/3/library/sqlite3.html
[2]: https://www.python.org/dev/peps/pep-0249/
[3]: https://wiki.python.org/moin/DatabaseInterfaces

## Table of Contents


[Python and SQLite](#Python-and-SQLite)

- [Python: Database Connections](#Python:-Database-Connections)
- [Database Cursor](#Database-Cursor)
- [Executing SQL Commands](#Executing-SQL-Commands)
- [Parameterized Queries](#Parameterized-Queries)
- [User Defined Functions](#User-Defined-Functions)

[Python Database Programming](#Python-Database-Programming)

[Pandas and SQL](#Pandas-and-SQL)

-----
[[Back to TOC]](#Table-of-Contents)

## Python and SQLite

To use the SQLite database from within a Python program, we simply need to import the sqlite3 module. Since SQLite is distributed with Python, there are no extra download or installation steps. If you wish to use a different database, you will first need to download the appropriate Python library for that database; and, second, install the library to the appropriate system path.

In the following Code cell, we demonstrate how to use SQLite from within a Python program. The first step is to simply import the sqlite3 module, after this we can access the module to interact with a database. In the first example, we simply access several module attributes that specify the version information for both the SQLIte database and sqlite3 module we currently have installed.

-----

In [1]:
import sqlite3 as sql

print(f'SQLite library version: {sql.version}')
print(f'SQLite version: {sql.sqlite_version}')

SQLite library version: 2.6.0
SQLite version: 3.13.0


-----

Since the `sqlite3` module is part of the standard Python library, this import process should be painless. We also can take advantage of the fact that the SQLite database itself is embedded within the Python interpreter. As a result, we can easily view the built-in help information for either the sqlite3 module or for specific attributes, functions, or types that are in the sqlite3 module by using the Python `help()` function.

-----

In [2]:
# View built-in help for module

help(sql)

Help on package sqlite3:

NAME
    sqlite3

MODULE REFERENCE
    https://docs.python.org/3.6/library/sqlite3
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    # pysqlite2/__init__.py: the pysqlite2 package.
    #
    # Copyright (C) 2005 Gerhard Häring <gh@ghaering.de>
    #
    # This file is part of pysqlite.
    #
    # This software is provided 'as-is', without any express or implied
    # warranty.  In no event will the authors be held liable for any damages
    # arising from the use of this software.
    #
    # Permission is granted to anyone to use this software for any purpose,
    # including commercial applications, and to alter it and redistribute it
    # freely, subject to the followi

In [3]:
# We can also get useful information on specific functions/types
help(sql.connect)

Help on built-in function connect in module _sqlite3:

connect(...)
    connect(database[, timeout, detect_types, isolation_level,
            check_same_thread, factory, cached_statements, uri])
    
    Opens a connection to the SQLite database file *database*. You can use
    ":memory:" to open a database connection to a database that resides in
    RAM instead of on disk.



-----
[[Back to TOC]](#Table-of-Contents)

## Python: Database Connections

To work with a database from within a Python program, we follow a specific set of operations:

1. Establish a _Connection_ to the database. With most database systems, this involves a [network connection][1] to a (possibly) remote database server. SQLite is an embedded database, however, so we, by default, simply need the database file to opened by the SQLite library. Note that we can also work with a temporary, in memory database, which will be demonstrated in the next few code blocks.

2. Obtain a _Cursor_ from the database connection. A [database cursor][2] is an object that enables us to programmatically move through a database table in order to insert, delete, update, or select data. 

3. Execute SQL commands by using the database cursor. These commands can be SQL DDL commands where we create schema, or SQL DML commands where we insert, update, delete, or select data. The execution process can return information, that we can programmatically use.

In the following code sample, we first establish a connection to an _in memory_ SQLite database. We do this by using a runtime context via the `with` statement. This will ensure that if the operations contained within the context code block are successful that database transaction will be committed. If there is a problem within the context, or an exception is thrown, however, the transaction will instead be rolled back. As a result, the database will be maintained in a consistent state.

We obtain a database connection by calling the `connect` built-in method in the sqlite3 library. The only required parameter is the name of the database, which is generally the full pathname to the database file, for example we could connect to our Docker container database by specifying the full path, for example, `/home/data_scientist/data/database/myds`. In this case, we use the special name `':memory:'`, which indicates that our database will be temporary and maintained in the program's memory space. This method returns an instance of the [SQLite Connection][3] object. 

```sql
with sql.connect(":memory:") as con:
```

-----

[1]: https://en.wikipedia.org/wiki/Database_connection
[2]: https://en.wikipedia.org/wiki/Cursor_(databases)
[3]: https://docs.python.org/3/library/sqlite3.html#connection-objects

[[Back to TOC]](#Table-of-Contents)

### Database Cursor

To interact with a database, we need a cursor. We can use an implicit cursor by calling execute methods directly on the connection object. The recommended approach, however, is to always be explicit, and in this case that means we use an explicit cursor. The following code sample demonstrates how to acquire a cursor from a database connection, `con`. The method will return an instance of the [SQLite Cursor][sc] object.

```sql
cur = con.cursor()    
```

-----
[sc]: https://docs.python.org/3/library/sqlite3.html#cursor-objects

[[Back to TOC]](#Table-of-Contents)

### Executing SQL Commands

Once we have a cursor object, we can begin to execute SQL commands. There are several methods that we can use, depending on our specific needs: 

- `execute()`: Will execute a single SQL statement. The SQL statement can be parameterized, in which case the replacement values also must be passed to the execute method.

- `executemany()`: Executes a SQL statement multiple times. The SQL statement, which is the first argument is parameterized and with each new invocation the parameters are replaced by the values contained in the second argument. This can be useful for inserting data into an existing table.

- `executescript()`: execute multiple SQL statements simultaneously. A transaction commit is first performed, and then the SQL statements contained in the argument passed into the method are evaluated.

After we have executed one or more SQL query statements, our cursor object will enable us to _fetch_ the results. There are three different fetch operations:

- `fetchone()`: returns the next row in the query result. The return data type is a single sequence containing the values in the row. If no data was returned, `None` is instead returned.

- `fetchmany()`: returns the next set of rows in the query result. The return data type is a list containing the individual rows (which are sequences). A size parameter can be supplied to indicate how many rows should be returned.

- `fetchall()`: returns all (remaining) rows of a query result. The return data type is a list, and if no more results remain, and empty list is returned.

-----

In [4]:
# Open a database connection, here we use an in memory DB
with sql.connect(":memory:") as con:

    # Now we obtain our cursor and execute a simple query.
    cur = con.cursor()    
    cur.execute('SELECT SQLITE_VERSION()')
    
    # Our simple query has one return value, so we only need to fetch one
    data = cur.fetchone()
    
    # Output the information
    print(f'SQLite version: {data[0]}')


SQLite version: 3.13.0


-----
[[Back to TOC]](#Table-of-Contents)

### Parameterized Queries

In the previous example, we 
1. established a database connection,
2. obtained our cursor,
3. executed a simple query to obtain the SQLite database version information, and
4. fetched the return value before displaying the result.

While trivial, this example does demonstrate how to perform the basic steps of working with a SQLite database. In the next example, however, we return to our Bigdog's Surf Shop example to programmatically create and populate a schema before issuing a query. In this case, we use a predefined SQL string to create our table, and use a tuple sequence to pass our data into the `executemany()` function. As shown in the example, the parameters are indicated in the SQL INSERT statement as question mark character format `?`. A second technique exists in which the parameters are explicitly named; this will be demonstrated later. The `executemany()` method replaces the parameters by the corresponding values from the `items` sequence, continuing until the sequence of items has been exhausted.

After the schema has been created and populated, we next issue a SQL query. In this case, we use the fact that the `execute()` method can be used as an iterator to iteratively process each row returned from our query. The `row` object is a sequence, which easily allows us to extract the desired columns.

-----

In [5]:
# We define our Create Table SQL command
createSQL = '''
CREATE TABLE myProducts (
    itemNumber INT NOT NULL,
    price REAL,
    stockDate TEXT,
    description TEXT);
'''

# Tuple containing data values to insert into our database
items = ((1,19.95,'2015-03-31','Hooded sweatshirt'), 
         (2,99.99,'2015-03-29','Beach umbrella'),
         (3,0.99,'2015-02-28', None),
         (4,29.95,'2015-02-10','Male bathing suit, blue'),
         (5,49.95,'2015-02-20','Female bathing suit, one piece, aqua'),
         (6,9.95,'2015-01-15','Child sand toy set'),
         (7,24.95,'2014-12-20','White beach towel'),
         (8,32.95,'2014-12-22','Blue-striped beach towel'),
         (9,12.95,'2015-03-12','Flip-flop'),
         (10,34.95,'2015-01-24','Open-toed sandal'))

# Open a database connection, here we use an in memory DB

with sql.connect(":memory:") as con:

    # Now we obtain our cursor
    cur = con.cursor()   
    
    # First we create the table
    cur.execute(createSQL)
    
    # Now populate the table using all items
    cur.executemany("INSERT INTO myProducts VALUES(?, ?, ?, ?)", items)

    # We can now select rows and iterate through them
    for row in cur.execute('SELECT * FROM myProducts'):
        print("{0} costs ${1}; last stocked on {2}.".format(row[3], row[1], row[2]))


Hooded sweatshirt costs $19.95; last stocked on 2015-03-31.
Beach umbrella costs $99.99; last stocked on 2015-03-29.
None costs $0.99; last stocked on 2015-02-28.
Male bathing suit, blue costs $29.95; last stocked on 2015-02-10.
Female bathing suit, one piece, aqua costs $49.95; last stocked on 2015-02-20.
Child sand toy set costs $9.95; last stocked on 2015-01-15.
White beach towel costs $24.95; last stocked on 2014-12-20.
Blue-striped beach towel costs $32.95; last stocked on 2014-12-22.
Flip-flop costs $12.95; last stocked on 2015-03-12.
Open-toed sandal costs $34.95; last stocked on 2015-01-24.


-----

The second approach to using parameterized SQL statements is to explicitly name the parameters. This is accomplished by using a colon `:` followed by the name in the SQL statement, and passing in a dictionary where the name used in the SQL statement is the key and the corresponding dictionary value holds the value to insert into the SQL statement. For example, we could replace the previous SQL INSERT statement with the equivalent named parameter form, using the `execute()` method:

```sql
for item in items:
    cur.execute("INSERT INTO myProducts VALUES(:id, :price, :sdate, :desc)", 
        {"id" : item[0], "price" : item[1], "sdate" : item[2], "desc" : item[3]})
```

-----
[[Back to TOC]](#Table-of-Contents)

### User Defined Functions

We can also create user-defined functions in SQLite, and use these new functions in a subsequent SQL Statement. This is demonstrated in the next Code cell, where we recreate our temporary database in memory, define three new functions to extract the _year_, _month_, and _day_ from the date text string in our database. To do this, we first create the Python versions of these functions: `myYear`, `myMonth`, and `myDay`. Next, we call the `create_function` method on our database connection to create the SQLite functions. 

This process essentially creates a map between the SQLite name of the function, which is used in SQL statements and the Python function. We also must pass in the expected number of arguments, which for all three of these demo-functions is simply the one data text string. Finally, we use these new functions in our SQL query, as we now explicitly return the year, month, and day separately for each item in the database. To be clear, these functions are simply for demonstration. To actually perform this operation, we should use regular expressions to build a more robust date component extraction.

-----

In [6]:
# We define three simple functions

def myYear(date):
    return int(date[:4])

def myMonth(date):
    return int(date[5:7])

def myDay(date):
    return (int(date[-2:]))

# Define our SQL query

query = 'SELECT description, fMonth(stockDate), fDay(stockDate), ' + \
        'fYear(stockDate) FROM myProducts'

# Open a database connection, here we use an in memory DB

with sql.connect(":memory:") as con:
    
    # Now we obtain our cursor
    cur = con.cursor()   
    
    # First we create the table
    cur.execute(createSQL)
    
    # Now populate the table using all items
    cur.executemany("INSERT INTO myProducts VALUES(?, ?, ?, ?)", items)
    
    # Create our user-defined functions
    con.create_function("fYear", 1, myYear)
    con.create_function("fMonth", 1, myMonth)
    con.create_function("fDay", 1, myDay)

    # We can now select rows by using our functions and iterate through results
    for row in cur.execute(query):
        print("{0} last stocked on {1}/{2}/{3}.".format(row[0], row[1], row[2], row[3]))

Hooded sweatshirt last stocked on 3/31/2015.
Beach umbrella last stocked on 3/29/2015.
None last stocked on 2/28/2015.
Male bathing suit, blue last stocked on 2/10/2015.
Female bathing suit, one piece, aqua last stocked on 2/20/2015.
Child sand toy set last stocked on 1/15/2015.
White beach towel last stocked on 12/20/2014.
Blue-striped beach towel last stocked on 12/22/2014.
Flip-flop last stocked on 3/12/2015.
Open-toed sandal last stocked on 1/24/2015.


-----

[[Back to TOC]](#Table-of-Contents)

## Python Database Programming

Now we can turn to executing SQL queries against an actual SQLite database. In this case, we will use a pre-built airline database. First, we define our database location, after which we  As shown below, the program logic is identical to the in memory database, with the exception that we now explicitly list the full path to our database file.

One important change, however, will be in the execution speed since this database is large and in a file in our Docker container as opposed to an in memory database.

-----

In [7]:
# First we find our HOME directory
home_dir = !echo $HOME

# Define data directory
database_dir = home_dir[0] +'/data/database/'

# Define database
database = database_dir + 'rppds'

print(f'Database is persisted at {database}\n')

Database is persisted at /home/data_scientist/data/database/rppds



-----

With the database name, we can now access this shared database from within our notebook. The following Code cell connects to this database and displays the number of flights for ten airplanes (defined by their tail number).

-----

In [8]:
# Test query to select ten airplanes and count their flights
query = "SELECT COUNT(*), tailNumber FROM flights GROUP BY tailNumber LIMIT 10"

with sql.connect(database) as con:
    
    cur = con.cursor()    
    
    for row in cur.execute(query):
        print(row)

(327, '-N037M')
(1170, '-N047M')
(1380, '-N107D')
(1376, '-N108D')
(1405, '-N109D')
(1387, '-N110D')
(1385, '-N110H')
(1485, '-N111D')
(1404, '-N112D')
(1424, '-N114D')


-----

We can also unpack the row returned from our SQL query directly into Python variables. This can simplify their subsequent use as demonstrated below.


-----

In [9]:
with sql.connect(database) as con:
    
    total = 0
    
    cur = con.cursor()    
    
    for row in cur.execute(query):
        count, number = row
        print(f'Flight# {number} flew {count} times in 2001')

        total += count
        
    print(f'\nWe found {total} total flights for the selected period.')

Flight# -N037M flew 327 times in 2001
Flight# -N047M flew 1170 times in 2001
Flight# -N107D flew 1380 times in 2001
Flight# -N108D flew 1376 times in 2001
Flight# -N109D flew 1405 times in 2001
Flight# -N110D flew 1387 times in 2001
Flight# -N110H flew 1385 times in 2001
Flight# -N111D flew 1485 times in 2001
Flight# -N112D flew 1404 times in 2001
Flight# -N114D flew 1424 times in 2001

We found 12743 total flights for the selected period.


-----

We also can query a database, and write out results to a stream, such as a file. In the following code cell, we open a file for writing, establish a database connection, execute a query, and write a formatted version of each row returned from the query to the file.

-----

In [10]:
fout = open('airports.txt', 'w')

with sql.connect(database) as con:
    
    cur = con.cursor()    
    
    for row in cur.execute('SELECT * FROM airports LIMIT 100'):
        print(f'Airport ID {row[0]} is located in {row[1]}, {row[2]}.',
              file=fout)

fout.close()

In [11]:
!head airports.txt

Airport ID 00M is located in Thigpen , Bay Springs.
Airport ID 00R is located in Livingston Municipal, Livingston.
Airport ID 00V is located in Meadow Lake, Colorado Springs.
Airport ID 01G is located in Perry-Warsaw, Perry.
Airport ID 01J is located in Hilliard Airpark, Hilliard.
Airport ID 01M is located in Tishomingo County, Belmont.
Airport ID 02A is located in Gragg-Wade , Clanton.
Airport ID 02C is located in Capitol, Brookfield.
Airport ID 02G is located in Columbiana County, East Liverpool.
Airport ID 03D is located in Memphis Memorial, Memphis.


-----

Since SQLite supports a limited set of data types: NULL, INTEGER, REAL, TEXT, and BLOB, we can face difficulties in trying to effectively map complex Python data types into SQLite data types. By default, the following mappings exist:

| Python | SQLite |
| -- | -- |
| None | NULL |
| int | INTEGER |
| float | REAL |
| str | TEXT |
| bytes | BLOB |

The only simply exception is that SQLite TEXT data can be converted to different Python types by using a [`text_factory`][1]. For example, the TEXT data can be returned as a [`byte`][2] string. To extend the SQLite default type system we can either use [object adaption][3] to store more Python data types in a SQLite database, or we can supply [converters][4] to transform SQLite data types into different Python types. Both of these are fairly specific to SQLite, however, and thus we will not discuss these in more detail. To persist more advanced data structures or classes, we should look at object-relational mapping tools like [SQL Alchemy][sa].

-----
[1]: https://docs.python.org/3/library/sqlite3.html#sqlite3.Connection.text_factory
[2]: https://docs.python.org/3/library/functions.html#bytes
[3]: https://docs.python.org/3/library/sqlite3.html#using-adapters-to-store-additional-python-types-in-sqlite-databases
[4]: https://docs.python.org/3/library/sqlite3.html#converting-sqlite-values-to-custom-python-types
[sa]: http://www.sqlalchemy.org

[[Back to TOC]](#Table-of-Contents)


## Pandas and SQL
Pandas provides built-in support for executing a SQL query and retrieving the result as a DataFrame. This is demonstrated in the next Code cell, where we execute a SQL query on the airlines database. We select several columns, and for simplicity we restrict our query result to only ten rows by using the ANSI SQL `LIMIT` clause.

The Pandas method to execute a SQL statement is `read_sql`, and mimics in appearance other Panda methods for _reading_ data into a Pandas DataFrame. In this case, the method takes our SQL statement, database connection, and an optional parameter, `index_col` that we can use to specify which column in our result should be treated as an index column. Pandas will supply an auto-incrementing column if no column is explicitly supplied. To save space in the output display, we specify our own column in these examples.

-----

In [12]:
import pandas as pd

query = "SELECT code, airport, city, state, latitude, longitude FROM airports LIMIT 10 ;"

with sql.connect(database) as con:
    data = pd.read_sql(query, con, index_col ='code')
    
    print(data)

                   airport              city state   latitude   longitude
code                                                                     
00M               Thigpen        Bay Springs    MS  31.953765  -89.234505
00R   Livingston Municipal        Livingston    TX  30.685861  -95.017928
00V            Meadow Lake  Colorado Springs    CO  38.945749 -104.569893
01G           Perry-Warsaw             Perry    NY  42.741347  -78.052081
01J       Hilliard Airpark          Hilliard    FL  30.688012  -81.905944
01M      Tishomingo County           Belmont    MS  34.491667  -88.201111
02A            Gragg-Wade            Clanton    AL  32.850487  -86.611453
02C                Capitol        Brookfield    WI  43.087510  -88.177869
02G      Columbiana County    East Liverpool    OH  40.673313  -80.641406
03D       Memphis Memorial           Memphis    MO  40.447259  -92.226961


-----

In the next code cell, we use the column selection feature with a Pandas DataFrame to select only those rows that have airports in the state of Mississippi. We do this by selecting the `state` attribute of the DataFrame, which corresponds to the `state` column, and applying the appropriate Boolean condition.

-----

In [13]:
query = "SELECT code, airport, city, state, latitude, longitude FROM airports LIMIT 100 ;"

with sql.connect(database) as con:
    data = pd.read_sql(query, con, index_col ='code')
    
    print(data[data.state == 'MS'])

                     airport         city state   latitude  longitude
code                                                                 
00M                 Thigpen   Bay Springs    MS  31.953765 -89.234505
01M        Tishomingo County      Belmont    MS  34.491667 -88.201111
04M           Calhoun County    Pittsboro    MS  33.930112 -89.342852
06M         Eupora Municipal       Eupora    MS  33.534566 -89.312569
08M    Carthage-Leake County     Carthage    MS  32.761246 -89.530071
09M     Charleston Municipal   Charleston    MS  33.991502 -90.078145
0M6            Panola County   Batesville    MS  34.366774 -89.900089
0R0   Columbia-Marion County     Columbia    MS  31.297008 -89.812829


-----

Pandas also simplifies the insertion of new data into a SQL database. For this, we can simply take an existing Pandas DataFrame and call the `to_sql()` method. This method requires two parameters, the name of the database table, and the database connection. If the table does not exist, a new table will be created to match the DataFrame, including appropriate column names and data types. 

In [5]:
# We drop the table if it exists
!sqlite3 test "DROP TABLE  ILAirports ;"

Error: no such table: ILAirports


In the next two code blocks, we first query the `airports` table, and use Pandas to extract all airports in Illinois. We next insert this data back into our database as a new table called `ILAirports`. The following code block queries this new table and display the results for confirmation.

-----

In [15]:
# Creating table automatically works better if columns are explicitly listed.

query = "SELECT code, airport, city, state, latitude, longitude FROM airports ;"
with sql.connect(database) as con:
    data = pd.read_sql(query, con)

with sql.connect('test') as con:
    data[data.state == 'IL'].to_sql('ILAirports', con)

In [16]:
with sql.connect('test') as con:
    data = pd.read_sql('SELECT code, city, airport, latitude, longitude FROM ILAirports', 
                       con, index_col ='code')
    
    print(data[10:20])

                       city                airport   latitude  longitude
code                                                                    
3CK       Lake In The Hills      Lake In The Hills  42.206803 -88.323040
3K6   Troy/Marine/St. Louis    St Louis-Metro East  38.732909 -89.806567
3LC                 Lincoln           Logan County  40.158472 -89.334972
3LF              Litchfield   Litchfield Municipal  39.166353 -89.674897
3MY                  Peoria   Mt. Hawley Auxiliary  40.795259 -89.613403
9I0                  Havana        Havana Regional  40.221155 -90.022894
AJG               Mt Carmel    Mt Carmel Municipal  38.606547 -87.726694
ALN         Alton/St. Louis     St. Louis Regional  38.890291 -90.046043
ARR          Chicago/Aurora       Aurora Municipal  41.771929 -88.475659
BLV    Belleville/St. Louis  Scott AFB/MidAmerica   38.545179 -89.835184


<font color='red' size = '5'> Student Exercise </font>

Earlier in this notebook, we used the sqlite module to execute SQL queries. Now that you have run the cells in this notebook, go back to the relevant cells and make these changes. Be sure to understand how your changes impact the file input and output process.

3. Try creating the Surf Shop database as a persistent database (i.e., not in memory).
4. With the persistent Surf Shop database, execute queries to count the number of items in the store, and sort them into descending order by their description.
56. The airport example demonstrate how easy it is to use the `to_sql` function on a Pandas DataFrame. Using any data set you choose (e.g., the Adult data or the Auto MPG data), read the data into a DataFrame and persist in a new database that you have created. Using the database, find all missing values.

-----

## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional.

1. [Official][1] SQLite Python DB-API implementation
2. The [SQLIte Python Tutorial][2], is a bit old and somewhat out-of-date, but it provides useful perspective.
3. [PEP-249][3], which outlines the common database API in Python
4. Working with [SQLite databases][4] by using Pandas
-----

[1]: https://docs.python.org/3/library/sqlite3.html
[2]: http://zetcode.com/db/sqlitepythontutorial/
[3]: https://www.python.org/dev/peps/pep-0249/
[4]: https://www.dataquest.io/blog/python-pandas-databases/

**&copy; 2017: Robert J. Brunner at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode